In [77]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import datetime
from sqlalchemy import create_engine

### Stock market 

In [69]:
stocks_url = 'https://finance.yahoo.com/most-active?offset=0&count=50'
tables = pd.read_html(stocks_url)
stock_df = tables[0]
stock_df.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",27.58,0.26,+0.95%,55.69M,67.647M,29.831B,110.76,NaN
1,GE,General Electric Company,10.26,-0.06,-0.58%,50.255M,80.429M,89.476B,NaN,NaN
2,BAC,Bank of America Corporation,28.29,-0.33,-1.15%,48.569M,56.255M,268.987B,10.52,NaN
3,TEVA,Teva Pharmaceutical Industries Limited,11.44,-0.66,-5.45%,46.142M,11.647M,12.498B,NaN,NaN
4,F,Ford Motor Company,10.36,0.12,+1.17%,39.644M,43.777M,41.332B,13.33,NaN


In [70]:
# Remove trailing letter and then convert datatype on the series of 'Market Cap' and 'Volume'
market_cap_cleaned = stock_df['Market Cap'].str.rstrip('B')
volume_cleaned = stock_df['Volume'].str.rstrip('M')

market_cap_converted = market_cap_cleaned.astype(float)
volume_converted = volume_cleaned.astype(float)

stock_df.drop(labels = 'Market Cap', axis=1, inplace=True)
stock_df.drop(labels = 'Volume', axis=1, inplace=True)
stock_df['Market Cap ($Bil.)'] = market_cap_converted
stock_df['Volume (Mil.)'] = volume_converted

In [71]:
stock_df.head()

,Symbol,Name,Price (Intraday),Change,% Change,Avg Vol (3 month),PE Ratio (TTM),52 Week Range,Market Cap ($Bil.),Volume (Mil.)
0,AMD,"Advanced Micro Devices, Inc.",27.58,0.26,+0.95%,67.647M,110.76,NaN,29.831,55.690
1,GE,General Electric Company,10.26,-0.06,-0.58%,80.429M,NaN,NaN,89.476,50.255
2,BAC,Bank of America Corporation,28.29,-0.33,-1.15%,56.255M,10.52,NaN,268.987,48.569
3,TEVA,Teva Pharmaceutical Industries Limited,11.44,-0.66,-5.45%,11.647M,NaN,NaN,12.498,46.142
4,F,Ford Motor Company,10.36,0.12,+1.17%,43.777M,13.33,NaN,41.332,39.644


In [96]:
stock_df_filtered = stock_df[stock_df['Market Cap ($Bil.)'] > 100.0]
stock_df_filtered.head()
stock_df_filtered['Time'] = datetime.datetime.now()
stock_df_filtered.head()

stock_df_filtered.reset_index(inplace=True)
stock_df_filtered.drop(labels = 'index', axis=1, inplace=True)
stock_df_filtered

C:\Users\Tai\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Tai\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Symbol,Name,Price (Intraday),Change,% Change,Avg Vol (3 month),PE Ratio (TTM),52 Week Range,Market Cap ($Bil.),Volume (Mil.),Time
0,BAC,Bank of America Corporation,28.29,-0.33,-1.15%,56.255M,10.52,NaN,268.987,48.569,2019-05-15 21:16:45.319632
1,CMCSA,Comcast Corporation,42.49,-0.42,-0.98%,17.118M,16.10,NaN,192.853,37.269,2019-05-15 21:16:45.319632
2,BABA,Alibaba Group Holding Limited,177.60,2.76,+1.58%,13.289M,50.79,NaN,460.371,36.744,2019-05-15 21:16:45.319632
3,CSCO,"Cisco Systems, Inc.",52.44,0.42,+0.81%,19.948M,19.13,NaN,230.842,30.577,2019-05-15 21:16:45.319632
4,PFE,Pfizer Inc.,41.15,0.49,+1.21%,23.052M,21.14,NaN,228.457,28.467,2019-05-15 21:16:45.319632
5,AAPL,Apple Inc.,190.92,2.26,+1.20%,28.578M,16.06,NaN,878.438,26.545,2019-05-15 21:16:45.319632
6,MSFT,Microsoft Corporation,126.02,1.75,+1.41%,25.021M,28.00,NaN,965.669,24.723,2019-05-15 21:16:45.319632
7,INTC,Intel Corporation,45.62,0.45,+1.00%,23.206M,10.30,NaN,204.241,23.408,2019-05-15 21:16:45.319632
8,T,AT&T Inc.,31.25,0.16,+0.51%,29.389M,11.78,NaN,228.062,20.396,2019-05-15 21:16:45.319632
9,WFC,Wells Fargo & Company,45.84,-0.65,-1.40%,22.541M,10.14,NaN,206.021,18.573,2019-05-15 21:16:45.319632


In [58]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)
# time.sleep(10)

In [59]:
# news_url = 'https://finance.yahoo.com/news/'
# browser.visit(news_url)

In [78]:
# browser.fill('input', stock_df.Symbol[0])
print(datetime.datetime.now())

2019-05-15 20:54:36.103605


In [99]:
rds_connection_string = "root:password@localhost"
engine = create_engine(f'mysql://{rds_connection_string}')

In [100]:
engine.execute('CREATE DATABASE IF NOT EXISTS wolves_db')

In [101]:
engine.execute('USE wolves_db')

In [102]:
stock_df_filtered.to_sql(name='market_data', con=engine, if_exists='append', index=False)